In [8]:
!pip install evaluate
!pip install rouge_score

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=53caa516eae09ccb4cbe6e1eb841a086ec9fa7e571921f3fa45f24411add5230
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


In [13]:
from datasets import load_dataset
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, Trainer, TrainingArguments
from transformers import DataCollatorWithPadding
import torch
import numpy as np
import evaluate



In [3]:

dataset = load_dataset("joelniklaus/legal_case_document_summarization", split="train[:5000]")  # Sample subset for quick training



README.md:   0%|          | 0.00/2.60k [00:00<?, ?B/s]

Repo card metadata block was not found. Setting CardData to empty.


train.jsonl.xz:   0%|          | 0.00/50.6M [00:00<?, ?B/s]

test.jsonl.xz:   0%|          | 0.00/2.30M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7773 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/200 [00:00<?, ? examples/s]

In [4]:
label_list = list(set(dataset["dataset_name"]))
label2id = {label: i for i, label in enumerate(label_list)}
id2label = {i: label for label, i in label2id.items()}
dataset = dataset.map(lambda e: {"label": label2id[e["dataset_name"]]})

tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")
model = DistilBertForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=len(label_list),
    id2label=id2label,
    label2id=label2id
)

def tokenize(batch):
    return tokenizer(batch["judgement"], truncation=True)

dataset = dataset.map(lambda e: {"label": int(label2id[e["dataset_name"]])})

dataset = dataset.map(tokenize, batched=True)

dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)



Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [5]:
split_dataset = dataset.train_test_split(test_size=0.1)
train_dataset = split_dataset["train"]
eval_dataset = split_dataset["test"]

In [6]:
accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return accuracy.compute(predictions=preds, references=labels)

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="no",
    per_device_train_batch_size=8,
    num_train_epochs=2,
    logging_dir="./logs",
    logging_steps=10,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,  
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

trainer.train()

model.save_pretrained("legal-bert-classifier")
tokenizer.save_pretrained("legal-bert-classifier")


/tmp/ipykernel_31/3388021717.py:22: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


RuntimeError: Found dtype Long but expected Float

In [8]:
dataset

Dataset({
    features: ['judgement', 'dataset_name', 'summary', 'label', 'input_ids', 'attention_mask'],
    num_rows: 5000
})

In [9]:
from datasets import load_dataset
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, Trainer, TrainingArguments
from transformers import DataCollatorWithPadding
import numpy as np
import evaluate

dataset = load_dataset("joelniklaus/legal_case_document_summarization", split="train[:1000]")  # keep small for dev

label_list = list(set(dataset["dataset_name"]))
label2id = {label: i for i, label in enumerate(label_list)}
id2label = {i: label for label, i in label2id.items()}

dataset = dataset.map(lambda e: {"label": int(label2id[e["dataset_name"]])})

tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")

def tokenize(example):
    return tokenizer(example["judgement"], truncation=True)

dataset = dataset.map(tokenize, batched=True)

split_dataset = dataset.train_test_split(test_size=0.1)
train_dataset = split_dataset["train"]
eval_dataset = split_dataset["test"]

train_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])
eval_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])

model = DistilBertForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=len(label_list),
    id2label=id2label,
    label2id=label2id
)

accuracy = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return accuracy.compute(predictions=preds, references=labels)

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="no",
    per_device_train_batch_size=8,
    num_train_epochs=2,
    logging_dir="./logs",
    report_to="none"
    
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics
)

trainer.train()

model.save_pretrained("legal-bert-classifier")
tokenizer.save_pretrained("legal-bert-classifier")


Repo card metadata block was not found. Setting CardData to empty.
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipykernel_31/1515684950.py:62: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


RuntimeError: Found dtype Long but expected Float

In [2]:
from datasets import load_dataset

dataset = load_dataset("joelniklaus/legal_case_document_summarization")
print(dataset["train"][0].key(), dataset["train"][0].value())  # Check structure: {'document': '...', 'summary': '...'}

Repo card metadata block was not found. Setting CardData to empty.


{'judgement': 'Appeal No. LXVI of 1949.\nAppeal from the High Court of judicature, Bombay, in a reference under section 66 of the Indian Income tax Act, 1022.\nK.M. Munshi (N. P. Nathvani, with him), for the appel lant. \' M.C. Setalvad, Attorney General for India (H. J. Umrigar, with him), for the respondent. 1950.\nMay 26.\nThe judgment of the Court was delivered by MEHR CHAND MAHAJAN J.\nThis is an appeal against a judgment of the High Court of Judicature at Bombay in an income tax matter and it raises the question whether munici pal property tax and urban immoveable property tax payable under the relevant Bombay Acts are allowable deductions under section 9 (1) (iv) of the Indian Income tax Act.\nThe assessee company is an investment company deriving its income from properties in the city of Bombay.\nFor the assessment year 1940 41 the net income of the assessee under the head "property" was computed by the Income tax Officer in the sum of Rs. 6,21,764 after deducting from gross re

In [4]:
print(dataset["train"][0].keys(), dataset["train"][0].values())  # Check structure: {'document': '...', 'summary': '...'}


dict_keys(['judgement', 'dataset_name', 'summary']) dict_values(['Appeal No. LXVI of 1949.\nAppeal from the High Court of judicature, Bombay, in a reference under section 66 of the Indian Income tax Act, 1022.\nK.M. Munshi (N. P. Nathvani, with him), for the appel lant. \' M.C. Setalvad, Attorney General for India (H. J. Umrigar, with him), for the respondent. 1950.\nMay 26.\nThe judgment of the Court was delivered by MEHR CHAND MAHAJAN J.\nThis is an appeal against a judgment of the High Court of Judicature at Bombay in an income tax matter and it raises the question whether munici pal property tax and urban immoveable property tax payable under the relevant Bombay Acts are allowable deductions under section 9 (1) (iv) of the Indian Income tax Act.\nThe assessee company is an investment company deriving its income from properties in the city of Bombay.\nFor the assessment year 1940 41 the net income of the assessee under the head "property" was computed by the Income tax Officer in th

In [5]:
from transformers import AutoTokenizer

model_name = "facebook/bart-base" 
tokenizer = AutoTokenizer.from_pretrained(model_name)

def preprocess_function(examples):
    inputs = tokenizer(
        examples["judgement"],
        truncation=True,
        padding="max_length",
        max_length=512  
    )
    labels = tokenizer(
        examples["summary"],
        truncation=True,
        padding="max_length",
        max_length=128  
    )
    return {
        "input_ids": inputs["input_ids"],
        "attention_mask": inputs["attention_mask"],
        "labels": labels["input_ids"]  
    }

tokenized_datasets = dataset.map(preprocess_function, batched=True)

config.json:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Map:   0%|          | 0/7773 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [14]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

import evaluate
rouge = evaluate.load("rouge")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(
        predictions=decoded_preds, references=decoded_labels, use_stemmer=True
    )
    return {k: round(v * 100, 4) for k, v in result.items()}

training_args = Seq2SeqTrainingArguments(
    output_dir="./legal_summarization",
    eval_strategy="epoch",
    per_device_train_batch_size=4,  
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    logging_steps=10,
    save_strategy="epoch",
    predict_with_generate=True,  
    report_to="none",
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,  
)

trainer.train()

/tmp/ipykernel_31/866388381.py:34: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,2.211300,2.171402,16.480600,8.063300,14.157000,15.218400
2,2.038300,2.105984,16.841000,8.413200,14.378600,15.578500
3,1.889100,2.082492,16.442000,7.973200,14.057700,15.093300


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3339: UserWarning: Moving the following attributes in the config to the generation config: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


TrainOutput(global_step=2916, training_loss=2.13913892882024, metrics={'train_runtime': 1552.4877, 'train_samples_per_second': 15.02, 'train_steps_per_second': 1.878, 'total_flos': 7109222234849280.0, 'train_loss': 2.13913892882024, 'epoch': 3.0})

In [19]:
from transformers import pipeline

summarizer = pipeline(
    "summarization",
    model="./legal_summarization/checkpoint-2916",  
    tokenizer=tokenizer,
)

sample_judgement = dataset["test"][0]["judgement"]  
summary = summarizer(sample_judgement, max_length=128)
print(summary[0]["summary_text"])

Device set to use cuda:0


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [16]:
summarizer = pipeline(
    "summarization",
    model=model.cpu(),
    tokenizer=tokenizer,
    device=-1,
)
summary = summarizer(sample[:2000])  # Test with shorter text first

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [11]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['judgement', 'dataset_name', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 7773
    })
    test: Dataset({
        features: ['judgement', 'dataset_name', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 200
    })
})

In [20]:
from transformers import pipeline
import torch

summarizer = pipeline(
    "summarization",
    model=model,
    tokenizer=tokenizer,
    device=0 if torch.cuda.is_available() else -1,
)

sample = dataset["test"][0]["judgement"]
try:
    summary = summarizer(
        sample,
        max_length=128,
        min_length=30,
        do_sample=False,
        truncation=True 
    )
    print(summary[0]["summary_text"])
except RuntimeError as e:
    print("Error:", e)
    print("Input length:", len(tokenizer(sample)["input_ids"]))

Device set to use cuda:0
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Error: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.

Input length: 4382


In [21]:
!zip -r model.zip ./legal_summarization/checkpoint-2916


  adding: legal_summarization/checkpoint-2916/ (stored 0%)
  adding: legal_summarization/checkpoint-2916/model.safetensors

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


 (deflated 8%)
  adding: legal_summarization/checkpoint-2916/special_tokens_map.json (deflated 52%)
  adding: legal_summarization/checkpoint-2916/training_args.bin (deflated 52%)
  adding: legal_summarization/checkpoint-2916/vocab.json (deflated 59%)
  adding: legal_summarization/checkpoint-2916/generation_config.json (deflated 45%)
  adding: legal_summarization/checkpoint-2916/merges.txt (deflated 53%)
  adding: legal_summarization/checkpoint-2916/config.json (deflated 64%)
  adding: legal_summarization/checkpoint-2916/trainer_state.json (deflated 81%)
  adding: legal_summarization/checkpoint-2916/tokenizer_config.json (deflated 75%)
  adding: legal_summarization/checkpoint-2916/rng_state.pth (deflated 25%)
  adding: legal_summarization/checkpoint-2916/tokenizer.json (deflated 82%)
  adding: legal_summarization/checkpoint-2916/optimizer.pt (deflated 8%)
  adding: legal_summarization/checkpoint-2916/scheduler.pt (deflated 56%)


In [22]:
from transformers import pipeline
import torch

summarizer = pipeline(
    "summarization",
    model="./legal_summarization/checkpoint-2916",
    tokenizer=tokenizer,
    device=0 if torch.cuda.is_available() else -1,
)

def safe_summarize(text, chunk_size=512):
    tokens = tokenizer(text, return_offsets_mapping=True, truncation=False)
    chunks = []
    current_chunk = []
    current_length = 0
    
    for i, (input_id, (start, end)) in enumerate(zip(tokens["input_ids"], tokens["offset_mapping"])):
        if current_length >= chunk_size:
            chunks.append(current_chunk)
            current_chunk = []
            current_length = 0
        current_chunk.append((start, end))
        current_length += 1
    
    if current_chunk:
        chunks.append(current_chunk)
    
    summaries = []
    for chunk in chunks:
        chunk_text = text[chunk[0][0]:chunk[-1][1]]
        try:
            summary = summarizer(chunk_text, max_length=128, min_length=30, truncation=True)
            summaries.append(summary[0]["summary_text"])
        except Exception as e:
            print(f"Error processing chunk: {e}")
            continue
    
    return " ".join(summaries)

long_text = dataset["test"][0]["judgement"]
summary = safe_summarize(long_text)
print(summary)

Device set to use cuda:0


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
